In [12]:
import pandas as pd
from itables import show
from IPython.display import HTML

In [14]:
def display_multiline(df, max_rows=20):
    """
    Display a DataFrame with automatic multiline wrapping:
    - Converts \n and \\n into HTML <br> for proper line breaks
    - Makes table cells wrap normally (no nowrap)
    - Limits displayed rows for readability
    """

    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.max_columns', None)

    df_disp = df.head(max_rows).copy()

    # Convert \n to <br> for multiline rendering
    def _ml(x):
        if isinstance(x, str):
            return x.replace("\\n", "<br>").replace("\n", "<br>")
        return x

    for col in df_disp.columns:
        df_disp[col] = df_disp[col].astype(str).apply(_ml)

    # Return a Styler that forces normal wrapping
    return df_disp.style.set_properties(**{
        "white-space": "normal",
        "word-wrap": "break-word"
    })

In [18]:
df = pd.read_csv("data/results/freshQA_since24_full.csv")
display_multiline(df[["model_response_agent_v1"]], max_rows=50)


,model_response_agent_v1
0,"1) Prerequisite You Were Checking: - I was verifying the definition/status of the current seat distribution in the U.S. Senate to determine by how many seats the Republicans control the Senate.2) Evidence-Based Status: - Verified. The evidence confirms that the Republicans hold 53 seats, while the Democrats hold 45 seats, with 2 independents caucusing with the Democrats.3) Updated Fact Ladder: - Identity: Who currently holds the majority in the United States Senate? [Verified] - Definition / Status: By how many seats does the majority party control the Senate? [Verified]4) Next Step: - All prerequisites are now verified. I will proceed to provide the Final Answer using the Answer Contract.Premise: TrueVerdict: NoDirect Answer: Democrats do not currently control the United States Senate; Republicans hold the majority.Key Facts:- Republicans hold 53 seats in the Senate, while Democrats hold 45 seats, with 2 independents caucusing with them (source: en.wikipedia.org, www.senate.gov).- This gives Republicans a majority by 6 seats over the Democrats and independents combined."
1,"Premise: Uncertain - The evidence does not confirm who is the current CEO of X Corp as of November 2025. It only confirms that Linda Yaccarino stepped down in July 2025.Verdict: UncertainDirect Answer: The current CEO of X Corp as of November 2025 is not confirmed based on the available evidence.Key Facts:- Linda Yaccarino stepped down as CEO of X Corp in July 2025 (source: Fortune, July 10, 2025).- No successor has been confirmed in the available evidence.If Needed: Further investigation or updated sources are required to determine the current CEO of X Corp."
2,"1) Prerequisite You Were Checking: - I was checking if Joe Biden has been convicted of any federal crimes.2) Evidence-Based Status: - Still Unclear. The evidence provided does not mention any federal crime convictions for Joe Biden. It discusses clemency actions and other legal matters but does not confirm any convictions.3) Updated Fact Ladder: - Identity: Determine who the most recent former President of the United States is as of today. [Verified] - Status: Check if Joe Biden has been convicted of any federal crimes. [Unresolved] - Definition / Status: Identify the specific federal crimes, if any, Joe Biden has been convicted of. [Unresolved]4) Next Step: - Given the lack of clear evidence, I will conclude that there is no verified information about Joe Biden being convicted of federal crimes. I will now provide the final answer based on the available evidence.Final Answer — Answer Contract:- Premise: Uncertain (There is no verified evidence that Joe Biden, the most recent former President, has been convicted of any federal crimes.)- Verdict: Uncertain- Direct Answer: There is no verified information that Joe Biden has been convicted of any federal crimes.-"
3,"Premise: Uncertain - The evidence does not clearly specify the exact month in 2021 when Alphabet's market capitalization reached its highest-ever recorded value.Verdict: UncertainDirect Answer: The specific month in 2021 when Alphabet's market capitalization reached its highest-ever recorded value is not clearly identified in the available evidence.Key Facts:- Evidence indicates Alphabet's market capitalization peaked on November 19, 2021, but does not confirm if this was the highest-ever recorded value. (source: investors.com)- Alphabet surpassed a $3 trillion market capitalization, but the timing of this milestone is not specified for 2021. (source: gov.ca.gov)If Needed: Further detailed financial records or reports from 2021 would be needed to confirm the exact month of the highest-ever market capitalization value for Alphabet."
4,"1) Prerequisite You Were Checking: - Confirm that Michael van Gerwen participated in the 2025 PDC World Darts Championship.2) Evidence-Based Status: - Contradicted. The evidence suggests that Michael van Gerwen did not participate in the 2025 PDC World Darts Championship due to p